## Manipulating fake sales data from a bookstore

##### In this repository we will do a slightly more complex exercise. We will create a fake datalake from a bookstore. 

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F 

In [4]:
spark = (
    SparkSession
    .builder
    .master("local[*]")
    .appName("Spark_Dataframe")
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
24/05/02 19:39:44 WARN Utils: Your hostname, DESKTOP-VMHJUON resolves to a loopback address: 127.0.1.1; using 192.168.92.85 instead (on interface eth0)
24/05/02 19:39:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/02 19:39:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
#create a clientes dataframe
clientes_df = (
    spark
    .read
    .option('delimiter', ';')
    .option('header', True)
    .option('inferSchema', True)
    .option('encoding', 'ISO-8859-1')
    .csv('lake/clientes.csv')
)

In [ ]:
#create a vendedores dataframe
vendedores_df = (
    spark
    .read
    .option('delimiter', ';')
    .option('header', True)
    .option('inferSchema', True)
    .option('encoding', 'ISO-8859-1')
    .csv('lake/vendedores.csv')
)

In [ ]:
#create a clientes dataframe
transacoes_df = (
    spark
    .read
    .option('delimiter', ';')
    .option('header', True)
    .option('inferSchema', True)
    .option('encoding', 'ISO-8859-1')
    .csv('lake/transacoes.csv')
)

In [ ]:
#create a clientes dataframe
livros_df = (
    spark
    .read
    .option('delimiter', ';')
    .option('header', True)
    .option('inferSchema', True)
    .option('encoding', 'ISO-8859-1')
    .csv('lake/livros.csv')
)

In [29]:
#show the schema of the dataframe
df.printSchema()

root
 |-- Regiao - Sigla: string (nullable = true)
 |-- Estado - Sigla: string (nullable = true)
 |-- Municipio: string (nullable = true)
 |-- Revenda: string (nullable = true)
 |-- CNPJ da Revenda: string (nullable = true)
 |-- Nome da Rua: string (nullable = true)
 |-- Numero Rua: string (nullable = true)
 |-- Complemento: string (nullable = true)
 |-- Bairro: string (nullable = true)
 |-- Cep: string (nullable = true)
 |-- Produto: string (nullable = true)
 |-- Data da Coleta: string (nullable = true)
 |-- Valor de Venda: string (nullable = true)
 |-- Valor de Compra: string (nullable = true)
 |-- Unidade de Medida: string (nullable = true)
 |-- Bandeira: string (nullable = true)



### Mandling data formats

Note in the schema that the column` 'Valor de Venda'` and `'Valor de Compra'`  is a string and follows the Brazilian pricing standard (BRL).

Then we will use the `regex_replace` method to replace the commas (, ) with points(.).
And the `cast` method to change from string to float.

In [30]:
# replace the ',' to '.' and cast the column 'Valor de Venda' to float
df = df.withColumn('Valor de Venda',  F.regexp_replace('Valor de Venda', ',', '.').cast('float'))
df = df.withColumn('Valor de Compra',  F.regexp_replace('Valor de Compra', ',', '.').cast('float'))
#show the schema of the dataframe for verificate the changes
df.printSchema()

root
 |-- Regiao - Sigla: string (nullable = true)
 |-- Estado - Sigla: string (nullable = true)
 |-- Municipio: string (nullable = true)
 |-- Revenda: string (nullable = true)
 |-- CNPJ da Revenda: string (nullable = true)
 |-- Nome da Rua: string (nullable = true)
 |-- Numero Rua: string (nullable = true)
 |-- Complemento: string (nullable = true)
 |-- Bairro: string (nullable = true)
 |-- Cep: string (nullable = true)
 |-- Produto: string (nullable = true)
 |-- Data da Coleta: string (nullable = true)
 |-- Valor de Venda: float (nullable = true)
 |-- Valor de Compra: float (nullable = true)
 |-- Unidade de Medida: string (nullable = true)
 |-- Bandeira: string (nullable = true)



`|-- Valor de Venda: float (nullable = true)`

`|-- Valor de Compra: float (nullable = true)`

In [31]:
#create a view to use in sql queries
df.createOrReplaceTempView('combustiveis')


In [32]:
# show the first 5 rows of the dataframe with the sql query
spark.sql('''
          SELECT `Estado - Sigla`, `Produto`, `Valor de Venda`, `Valor de Compra`, `Unidade de Medida` FROM combustiveis
          ''').show(5)

+--------------+------------------+--------------+---------------+-----------------+
|Estado - Sigla|           Produto|Valor de Venda|Valor de Compra|Unidade de Medida|
+--------------+------------------+--------------+---------------+-----------------+
|            AL|          GASOLINA|          6.08|           NULL|       R$ / litro|
|            AL|GASOLINA ADITIVADA|          6.08|           NULL|       R$ / litro|
|            AL|            ETANOL|          4.78|           NULL|       R$ / litro|
|            AL|          GASOLINA|          5.79|           NULL|       R$ / litro|
|            AL|            ETANOL|          4.29|           NULL|       R$ / litro|
+--------------+------------------+--------------+---------------+-----------------+
only showing top 5 rows



In [33]:
# show the rows where the column 'Valor de Compra' is not null
spark.sql('''
          SELECT * FROM combustiveis WHERE `Valor de Compra` IS NOT NULL
          ''').show()
# as all dataframe values ​​'Valor de Compra' are null, we will not work with it

+--------------+--------------+---------+-------+---------------+-----------+----------+-----------+------+---+-------+--------------+--------------+---------------+-----------------+--------+
|Regiao - Sigla|Estado - Sigla|Municipio|Revenda|CNPJ da Revenda|Nome da Rua|Numero Rua|Complemento|Bairro|Cep|Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|Bandeira|
+--------------+--------------+---------+-------+---------------+-----------+----------+-----------+------+---+-------+--------------+--------------+---------------+-----------------+--------+
+--------------+--------------+---------+-------+---------------+-----------+----------+-----------+------+---+-------+--------------+--------------+---------------+-----------------+--------+



In [34]:
#create a new dataframe with the sql query
view_precos = spark.sql(
    '''
 SELECT
     `Estado - Sigla`,
     `Produto`,
     `Valor de Venda`,
     `Unidade de Medida` 
     FROM combustiveis
    '''
)


In [35]:
view_precos.show(10)

+--------------+------------------+--------------+-----------------+
|Estado - Sigla|           Produto|Valor de Venda|Unidade de Medida|
+--------------+------------------+--------------+-----------------+
|            AL|          GASOLINA|          6.08|       R$ / litro|
|            AL|GASOLINA ADITIVADA|          6.08|       R$ / litro|
|            AL|            ETANOL|          4.78|       R$ / litro|
|            AL|          GASOLINA|          5.79|       R$ / litro|
|            AL|            ETANOL|          4.29|       R$ / litro|
|            AL|          GASOLINA|          5.89|       R$ / litro|
|            AL|GASOLINA ADITIVADA|          6.09|       R$ / litro|
|            AL|            ETANOL|          4.49|       R$ / litro|
|            AL|          GASOLINA|          5.89|       R$ / litro|
|            AL|GASOLINA ADITIVADA|          6.09|       R$ / litro|
+--------------+------------------+--------------+-----------------+
only showing top 10 rows



#### We will carry out a price analysis on the dataframe `'Valor de Venda'` that has had its metadata previously  treaties.

In [36]:
#create a view to use in sql queries
df.createOrReplaceTempView('view_precos')

#### Let's create a column `'diferenca'` where it will receive the subtraction of the highest and lowest sales value. And we will sort it in descending order.


In [37]:
df_diferenca_precos = spark.sql(
    """
     SELECT
     `Estado - Sigla`,
     `Produto`,
     `Unidade de Medida` ,
     MIN(`Valor de Venda`) as `Menor Preço`,
     MAX(`Valor de Venda`) as `Maior Preço`,
     MAX(`Valor de Venda`) - MIN(`Valor de Venda`) as diferenca
     FROM view_precos
     GROUP BY ALL
     ORDER BY  diferenca DESC
    """
)

In [38]:
df_diferenca_precos.show(5)

+--------------+------------------+-----------------+-----------+-----------+---------+
|Estado - Sigla|           Produto|Unidade de Medida|Menor Preço|Maior Preço|diferenca|
+--------------+------------------+-----------------+-----------+-----------+---------+
|            SP|GASOLINA ADITIVADA|       R$ / litro|       4.79|       8.69|3.8999996|
|            SP|            ETANOL|       R$ / litro|       2.78|       6.19|     3.41|
|            SP|          GASOLINA|       R$ / litro|       4.59|       7.59|      3.0|
|            PA|            ETANOL|       R$ / litro|       3.85|        6.6|     2.75|
|            RS|            ETANOL|       R$ / litro|       3.88|       6.29|2.4099998|
+--------------+------------------+-----------------+-----------+-----------+---------+
only showing top 5 rows

